<a href="https://colab.research.google.com/github/springboardmentor891v/Email_Assistant_Using_LangGraph/blob/manoj_preetham/Email_Assistant.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
pip install langgraph langchain langchain-openai google-auth-oauthlib google-api-python-client


In [12]:
import os
import base64
from typing import TypedDict, Annotated, List, Literal
from email.mime.text import MIMEText

from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage
from langgraph.graph import StateGraph, END
from langgraph.checkpoint.memory import MemorySaver

# Google API Imports
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from googleapiclient.discovery import build

In [15]:
# 1. SETUP GMAIL AUTHENTICATION
SCOPES = ['https://www.googleapis.com/auth/gmail.modify']

def get_gmail_service():
    creds = None

    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file(
            'token.json', SCOPES
        )

    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'credential.json',
                SCOPES,
                redirect_uri='urn:ietf:wg:oauth:2.0:oob'
            )

            auth_url, _ = flow.authorization_url(
                prompt='consent'
            )

            print("\n🔗 OPEN THIS URL IN YOUR BROWSER:\n")
            print(auth_url)

            code = input("\n📌 Paste the authorization code here: ")
            flow.fetch_token(code=code)
            creds = flow.credentials

        with open('token.json', 'w') as token:
            token.write(creds.to_json())

    return build('gmail', 'v1', credentials=creds)

service = get_gmail_service()


🔗 OPEN THIS URL IN YOUR BROWSER:

https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=915502763627-c2ilmm6t0n042r0lbjg7mjr0f0mmqehn.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fgmail.modify&state=vY3KKjajbADFw0NBXloebmYwUBTIcU&prompt=consent&access_type=offline

📌 Paste the authorization code here: 4/1ASc3gC0h2okDoeZ46LvVMJOmJREuDRFvyWjEIN24o8aixoIPe2LSQCzV4mg


In [20]:
import base64

def extract_email_data(service, msg_id):
    message = service.users().messages().get(
        userId='me',
        id=msg_id,
        format='full'
    ).execute()

    headers = message['payload']['headers']
    subject = sender = date = ""

    for h in headers:
        if h['name'] == 'Subject':
            subject = h['value']
        elif h['name'] == 'From':
            sender = h['value']
        elif h['name'] == 'Date':
            date = h['value']

    # Extract body
    body = ""
    payload = message['payload']

    if 'parts' in payload:
        for part in payload['parts']:
            if part['mimeType'] == 'text/plain':
                data = part['body'].get('data')
                if data:
                    body = base64.urlsafe_b64decode(data).decode('utf-8')
    else:
        data = payload['body'].get('data')
        if data:
            body = base64.urlsafe_b64decode(data).decode('utf-8')

    return subject, sender, date, body


# 🔁 Read the emails you listed
for msg in messages:
    msg_id = msg['id']
    subject, sender, date, body = extract_email_data(service, msg_id)

    print("📧 SUBJECT:", subject)
    print("👤 FROM:", sender)
    print("📅 DATE:", date)
    print("📝 BODY (preview):", body[:300])
    print("-" * 60)

📧 SUBJECT: j,hdq
👤 FROM: Chandan <chandankp034@gmail.com>
📅 DATE: Tue, 20 Jan 2026 21:58:12 +0530
📝 BODY (preview): 

------------------------------------------------------------


In [18]:
results = service.users().messages().list(
    userId='me',
    labelIds=['UNREAD'],
    maxResults=1000
).execute()

messages = results.get('messages', [])
print(f"📬 Unread emails found: {len(messages)}")

📬 Unread emails found: 1


In [23]:
import os
from openai import OpenAI

# Google Colab specific import for secrets
from google.colab import userdata

# Read API key from Colab secrets
api_key = userdata.get("OPENAI_API_KEY")

if not api_key:
    raise ValueError("OPENAI_API_KEY not found in Colab secrets. Please ensure it's set in the Colab 'Secrets' tab.")

# Create OpenAI client
client = OpenAI(api_key=api_key)

try:
    # List models available for this key
    models = client.models.list()

    print("Models available for this key:")
    for model in models.data:
        print(model.id)

except Exception as e:
    print("Error:", e)

Models available for this key:
gpt-3.5-turbo
gpt-5.2-codex
gpt-4o-mini-tts-2025-12-15
gpt-realtime-mini-2025-12-15
gpt-audio-mini-2025-12-15
chatgpt-image-latest
davinci-002
babbage-002
gpt-3.5-turbo-instruct
gpt-3.5-turbo-instruct-0914
dall-e-3
dall-e-2
gpt-3.5-turbo-1106
tts-1-hd
tts-1-1106
tts-1-hd-1106
text-embedding-3-small
text-embedding-3-large
gpt-3.5-turbo-0125
gpt-4o
gpt-4o-2024-05-13
gpt-4o-mini-2024-07-18
gpt-4o-mini
gpt-4o-2024-08-06
gpt-4o-audio-preview
omni-moderation-latest
omni-moderation-2024-09-26
gpt-4o-audio-preview-2024-12-17
gpt-4o-mini-audio-preview-2024-12-17
o1-2024-12-17
o1
gpt-4o-mini-audio-preview
o3-mini
o3-mini-2025-01-31
gpt-4o-2024-11-20
gpt-4o-search-preview-2025-03-11
gpt-4o-search-preview
gpt-4o-mini-search-preview-2025-03-11
gpt-4o-mini-search-preview
gpt-4o-transcribe
gpt-4o-mini-transcribe
gpt-4o-mini-tts
o3-2025-04-16
o4-mini-2025-04-16
o3
o4-mini
gpt-4.1-2025-04-14
gpt-4.1
gpt-4.1-mini-2025-04-14
gpt-4.1-mini
gpt-4.1-nano-2025-04-14
gpt-4.1-nano

In [24]:
! pip install langchain langgraph langchain-openai python-dotenv

In [25]:
import base64

def read_email(service, msg_id):
    msg = service.users().messages().get(
        userId='me',
        id=msg_id,
        format='full'
    ).execute()

    headers = msg['payload']['headers']
    subject = sender = ""

    for h in headers:
        if h['name'] == 'Subject':
            subject = h['value']
        if h['name'] == 'From':
            sender = h['value']

    body = ""
    payload = msg['payload']

    if 'parts' in payload:
        for part in payload['parts']:
            if part['mimeType'] == 'text/plain':
                data = part['body'].get('data')
                if data:
                    body = base64.urlsafe_b64decode(data).decode()
    else:
        data = payload['body'].get('data')
        if data:
            body = base64.urlsafe_b64decode(data).decode()

    return subject, sender, body

In [27]:
from openai import OpenAI
import os

# Use the api_key variable already loaded from Colab secrets
client = OpenAI(api_key=api_key)

def generate_reply(subject, sender, body):
    prompt = f"""
You are a professional email assistant.

Email Subject: {subject}
From: {sender}
Email:
{body}

Write a polite, clear, and professional reply.
"""

    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.4
    )

    return response.choices[0].message.content.strip()

In [28]:
from email.mime.text import MIMEText

def send_reply(service, to, subject, reply_text, thread_id):
    message = MIMEText(reply_text)
    message['to'] = to
    message['subject'] = "Re: " + subject

    raw = base64.urlsafe_b64encode(message.as_bytes()).decode()

    service.users().messages().send(
        userId='me',
        body={
            'raw': raw,
            'threadId': thread_id
        }
    ).execute()

In [30]:
for msg in messages:
    msg_id = msg['id']
    thread_id = msg['threadId']

    subject, sender, body = read_email(service, msg_id)

    print("\n" + "="*70)
    print("📧 SUBJECT:", subject)
    print("👤 FROM:", sender)
    print("📝 BODY:\n", body[:500])
    print("="*70)

    decision = input("❓ Do you want to reply to this email? (y/n): ").lower()

    if decision != 'y':
        print("⏭️ Skipped")
        continue

    # Step 1: Generate reply
    reply = generate_reply(subject, sender, body)

    print("\n🤖 Generated Reply:\n")
    print(reply)
    print("-"*70)

    approval = input("✅ Approve this reply? (y / n / edit): ").lower()

    if approval == 'edit':
        reply = input("\n✏️ Enter your edited reply:\n")

    if approval in ['y', 'edit']:
        send_reply(
            service,
            to=sender,
            subject=subject,
            reply_text=reply,
            thread_id=thread_id
        )
      # Mark as read
        service.users().messages().modify(
            userId='me',
            id=msg_id,
            body={'removeLabelIds': ['UNREAD']}
        ).execute()

        print("📤 Reply sent and email marked as read")

    else:
        print("❌ Reply not sent")


📧 SUBJECT: j,hdq
👤 FROM: Chandan <chandankp034@gmail.com>
📝 BODY:
 

❓ Do you want to reply to this email? (y/n): y

🤖 Generated Reply:

Subject: Re: j,hdq

Dear Chandan,

Thank you for your email. It seems that the subject line may have been entered incorrectly, and I would like to assist you with your inquiry. Could you please provide more details or clarify your message?

Looking forward to your response.

Best regards,

[Your Name]  
[Your Position]  
[Your Company]  
[Your Contact Information]
----------------------------------------------------------------------
✅ Approve this reply? (y / n / edit): y
📤 Reply sent and email marked as read
